In [ ]:
import pandas as pd
from IPython.display import clear_output

l = pd.read_csv("SE&TElist.csv")
emptylist = []
for i in l['name']:
    emptylist.append(0)
l['attendance'] = emptylist

log = ''
archivedNames = []

zoom = pd.read_csv("zoomlist.csv")
zoom = zoom.drop(['Guest','In Waiting Room','Recording Consent','Join Time','Leave Time'],axis = 'columns')

def addOrArchive(possibleid,tempname):
    global log
    tempstring = 'WARNING! Check this attendance carefully.'
    print(tempstring)
    log = log + tempstring + '\n'
    giveAttendance(possibleid[0][0])
#    for temp in range(len(l)):
#        if l['id'][temp] == possibleid[0][0]:
#            tempstring = l['class'][temp] +'\t'+ str(l['roll'][temp]) +'\t'+ l['name'][temp]
#            print(tempstring)
#            log = log + tempstring + '\n'
#            break
#    tempstring = 'Give attendance this person? '
#    tempstr = input(tempstring)
#    log = log + tempstring + tempstr + '\n'
#    if tempstr == '' or tempstr.upper()[0] != 'N':
#        giveAttendance(possibleid[0][0])
#    else:
#        tempstr = ''
#        tempstring = 'Archive this individual? '
#        tempstr = input(tempstring)
#        log = log + tempstring + tempstr + '\n'
#        if tempstr == '' or tempstr.upper()[0] != 'N':
#            archive(tempname)
#        else:
#            addOrArchive(possibleid,tempname)

def archive(name):
    global log
    archivedNames.append(name)
    tempstring = ' '.join(name[0]) + '\thas been archived.\n'
    print(tempstring)
    log = log + tempstring + '\n'
    
def choice(possibleid,name):
    global log
    templength = len(possibleid)
    tempstring = 'choose from the following:'
    print(tempstring)
    log = log + tempstring + '\n'
    for temp in range(templength):
        nametemp = ''
        for temp2 in range(len(l)):
            if l['id'][temp2] == possibleid[temp][0]:
                nametemp = l['name'][temp2] + '\t' + l['class'][temp2] + '\t' + str(l['roll'][temp2])
                break
        print(temp,':\t',nametemp,sep = '')
        log = log + str(temp) + ':\t' + nametemp + '\n'
    tempstring = str(templength) + ':\tarchive ' + ' '.join(name[0])
    print(tempstring)
    log = log + tempstring + '\n'
    tempstring = 'Enter your choice: '
    inp = input(tempstring)
    log = log + tempstring + inp + '\n'
    if inp.isdigit(): 
        inp = int(inp)
        if inp == templength:
            archive(name)
        elif inp > -1 and inp < templength:
            giveAttendance(possibleid[inp][0])
        else:
            choice(possibleid,name)
    else:
        choice(possibleid,name)

def giveAttendance(sid):
    global log
    for temp in range(len(l)):
        if l['id'][temp] == sid:
            l['attendance'][temp] = 1
            tempstring = 'Attendance given to:\t' + l['class'][temp] +'\t'+ str(l['roll'][temp]) +'\t'+ l['name'][temp] + '\n'
            print(tempstring)
            log = log + tempstring + '\n'
            break

srnumber = 0
emaillist = []
for i in range(len(zoom)):
    email = zoom['User Email'][i]
    if email in emaillist:
        continue
    srnumber = srnumber + 1
    emaillist.append(email)
    df = zoom.loc[zoom['User Email'] == email]
    namelist = []
    for name in df['Name (Original Name)']:
        if name in namelist:
            continue
        else:
            namelist.append(name)
    totaltime = 0
    for t in df['Duration (Minutes)']:
        totaltime = totaltime + t
    print('\n',str(srnumber),':',sep='')
    print("Now we have:\t", end = "\n")
    print(', '.join(namelist))
    print("Time stayed for:\t" , totaltime, " minutes", sep = "", end = "\n" )
    data = namelist[-1].upper().split('_')
    print(data)
    log = log + '\n' + str(srnumber) + ':\n'
    log = log + 'Now we have:\n'+', '.join(namelist)+"\nTime stayed for:\t"+str(totaltime)+" minutes\n"+' '.join(data)+'\n'
    year = 0
    clss = 0
    roll = 0
    name = []
    for tempstr1 in data:
        tempstr1 = tempstr1.split('-')
        for tempstr2 in tempstr1:
            tempstr2 = tempstr2.split('.')
            for tempstr3 in tempstr2:
                tempstr3 = tempstr3.split(' ')
                for s in tempstr3:
                    if s.replace(" ", "") in ['SE',]:
                        year = 1000
                    elif s.replace(" ", "") in ['TE',]:
                        year = 2000
                    elif s.replace(" ", "") in ['A','ITA',]:
                        clss = 100
                    elif s.replace(" ", "") in ['B','ITB',]:
                        clss = 200
                    elif s.replace(" ", "") in ['C','ITC',]:
                        clss = 300
                    elif s != '' and s[0].isdigit():
                        roll = int(s)
                    elif s!= 'IT' and s!= '': 
                        name.append(s.split(' '))
    sid = year + clss + roll
    if year != 0 and clss != 0 and roll != 0:
        giveAttendance(sid)   
    else:
        templist = []
        if clss == 0 and year == 0 and roll != 0:
            templist = [1100+sid,1200+sid,1300+sid,2100+sid,2200+sid,]
        elif clss == 0 and year != 0 and roll != 0:
            templist = [100+sid,200+sid,300+sid,]
        elif clss != 0 and year == 0 and roll != 0:
            templist = [1000+sid,2000+sid,]
        elif roll == 0 and year !=0 and clss !=0:
            templist = [(temp+year+clss) for temp in range(0,100)]
        elif roll == 0 and year == 0 and clss != 0:
            for temp in range(1,100):
                templist.append(temp + clss + 1000)
            for temp in range(1,100):
                templist.append(temp + clss + 2000)
        elif roll == 0 and year != 0 and clss == 0:
            if year == 1000: temp1 = 400
            elif year == 2000: temp1 = 300
            for temp2 in range(101,temp1):
                templist.append(year + temp2)
        elif roll == 0 and year == 0 and clss == 0:
            for temp in l['id']:
                templist.append(temp)
        possibleid = []
        for temp in templist:
            for temp2 in range(len(l)):
                if l['id'][temp2] == temp:
                    nametemp = l['name'][temp2].split(' ')
                    if nametemp[1] == name[0][0] or nametemp[2] == name[0][0]:
                        possibleid.append((temp,l['name'][temp2]),)
        if len(possibleid) == 1:
            addOrArchive(possibleid,name)
        elif len(possibleid) == 0:
            archive(name)
        else:
            choice(possibleid,name)

#clear_output()
tempstring = '\nN o w . g i v i n g . a t t e n d a n c e . t o . A C M c o r e.\n'
print(tempstring)
log = log + tempstring + '\n'
acmCore = [1164, 1244, 1325, 2132, 2162, 2226, 2250, 2251, 2254, 2255]
for temp in acmCore:
    giveAttendance(temp)

tempstring = '\n\nMoving onto archived names.\nArchived names:\n'
print(tempstring,sep='',end='')
log = log + tempstring + '\n'
for name in archivedNames:
    if name[0][0] == 'ACM':
        continue
    tempstring = ''
    for tempstr1 in name:
        tempstring = tempstring + ' '.join(tempstr1) + ' '
    print(tempstring)
    log = log + tempstring + '\n'

file = open('log.txt','w')
file.write(log)
file.close()

In [ ]:
l.to_csv('attendance-given.csv',index = False)

In [ ]:
print(log)